In [1]:
!pip install vidaug

In [12]:
from pathlib import Path
import os , os.path
from tkinter import W
from vidaug import augmentors as va
from cv2 import VideoWriter_fourcc, VideoWriter, VideoCapture
import random
from csv import writer
from shutil import rmtree
from numpy import ndarray

In [3]:
class Writer:
    temp_path = ''

    def write_to_csv(self, result: ndarray, out_csv: str):
        out_csv = Path(out_csv)

        with out_csv.open('a+') as csv:
            csv_writer = writer(csv)
            csv_writer.writerow(result)

    def write_as_video(self, path, image):
        if self.temp_path != path: 
            self.temp_path = path
            self.writer = VideoWriter(path, VideoWriter_fourcc('M','J','P','G'), 10, (640, 480))

        self.writer.write(image)


In [4]:
aug_size_per_class = 6

source = "/content/drive/MyDrive/rohit+vihang_videos/video"
out = "/content/drive/MyDrive/rohit+vihang_videos/aug_vid"
wrong_framelength_videos = "/content/drive/MyDrive/rohit+vihang_videos/aug_vid/failure"

source_ds = Path(source)
aug_ds = Path(out)
wrong_framelength_videos_ds = Path(wrong_framelength_videos)

random.seed(1)
vr = Writer()
print(source_ds.exists(),  aug_ds.exists(), wrong_framelength_videos_ds.exists())   #MAKE SURE THESE THREE ARE TRUE, 
                                                                                    #DONOT PROCEED IF ITS FALSE

True True True


# MAKE SURE ABOVE THREE ARE TRUE. IF NOT, THEN CREATE THAT PATH MANUALLY

In [5]:
classes = list(x.name for x in source_ds.glob('*'))
print(classes)

['please', 'Work', 'Movie']


In [6]:
all_source_ds = []
for category in classes:
    cat_dir = list((source_ds/category).glob("*"))
    all_source_ds.append(cat_dir)
counter_class = len(classes)

In [16]:
for i in range(len(classes)):    
    counter_vid = aug_size_per_class
    category = classes[i]
    current_ds = all_source_ds[i]
    print(category)
    make_dir = Path(out)
    make_dir = make_dir / category
    if make_dir.exists(): rmtree(make_dir)
    make_dir.mkdir()
    for _ in range(aug_size_per_class):
        cap = VideoCapture(str(random.choice(current_ds)))
        video_frames = []

        while True:
            status, frame = cap.read()
            if not status: break
            video_frames.append(frame)

        print(len(video_frames))
        video_shape = video_frames[0].shape

        seq = va.Sequential([va.RandomTranslate(int(video_shape[1]/4),50), va.RandomShear(0.07,0.07)])

        video_aug = seq(video_frames)
        aug_vid_name = aug_ds/category/f"{random.randint(100000,999999)}.avi"
        if len(video_aug) == 50:
            for aug_frame in video_aug:
                vr.write_as_video(str(aug_vid_name), aug_frame)
        else:
            print("WRONG FRAME LENGTH" + str(len(video_aug)) )
            aug_vid_name = wrong_framelength_videos/category/f"{random.randint(100000,999999)}.avi"
            for aug_frame in video_aug:
                vr.write_as_video(str(aug_vid_name), aug_frame)

        print("class:",counter_class,"||","vidno:",counter_vid,"/",aug_size_per_class,"||","filename:",aug_vid_name)
        counter_vid -=1

    counter_class -=1

Movie


It Takes Upto 10 Min To Produce 100 Augmented Videos, No GPU Required